### Objective - Event Registration Agent
- Allows users to navigate topics available in a seminar and to register

In [84]:
# Basic Imports
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [85]:
# Define Agent State
from typing import Annotated
from typing_extensions import TypedDict
import operator
from langgraph.graph.message import add_messages
from langchain_core.messages import  AnyMessage, SystemMessage, HumanMessage, ToolMessage

class RegisterState(TypedDict):
    """ State representing User interaction with Event Registration Agent that persists all conversations between human, agent and function calls. """
    messages: Annotated[list[AnyMessage], operator.add]
    registered_events : list[str]
    finished_conversation : bool


system_prompt = """ 
You are a helpful ai assistant that helps users register for events going to be held on June 1, 2025 at Hunt Valley, MD, USA - 21030. The name of the event is - Generative AI for Software Developers.
You will be provided with a list of events and their details.
You will ask the user for their name and email address.
You will then ask the user which events they would like to register for. Then You shall allow the user to enter one event at a time by calling the function add_registration
You will confirm with the user before registering them for the event, or allow them to change their mind by fetching the data added by calling get_registration.
If the user confirms the event you will register them by calling the function confirm_registration.
If the user wants to cancel the registration for an event, you will call the function delete_registration.
If the user wants to modify the registration for an event that the user had registered before, then just delete the previous registration by calling function delete_registration and create a new one by calling function add_registration.
You will then register the user for those events.
You will then ask the user if they would like to register for any other events.
If the user says no, you will end the conversation. After ending the convesation show a summary of registration by calling function summary_registration.
If you encounter a situation for which you do not have an answer or a function call, then simply return a response back to the user that this enhancement is not quite available, and will be available in future.
Keep all the conversation related to the registration of the event only. Do not engage into personal or emotional conversations.
"""
welcome_prompt = """ 
welcome to the registration agent for the Seminar - Generative AI for Software Developers. 
How can I help you today ? Please provide your email to register for the event. 
You shall get a chance to add/delete/modify your registration. To finish the chat simply type - quit
"""



In [86]:
# Define Chatbot
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Literal

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Define Agent Node
def agent(state: RegisterState) -> RegisterState:
    """ This is a simplle chatbot that can help with registration """
    message_history = [SystemMessage(content=system_prompt)] + state["messages"]
    return {"messages": [llm.invoke(message_history)]}

# Define Human Node
def human_node(state: RegisterState) -> RegisterState:
    """ Display last model message to user and receive user input """
    last_message = state["messages"][-1]
    print(f"Assistant: {last_message.content}")
    user_msg = input("User: ")
    if user_msg.lower() == "quit":
        state["finished_conversation"] = True
        return state
    return state | {"messages": [HumanMessage(content=user_msg)]}


def welcome_node(state: RegisterState) -> RegisterState:
    """ Display welcome message to user and receive user input """
    if state["messages"]:
        new_message = [SystemMessage(content=system_prompt)] + state["messages"]
    else:
        new_message = [SystemMessage(content=welcome_prompt)]
    return state | {"messages": new_message}

def user_quit_node(state: RegisterState) -> Literal["agent", "__end__"]:
    """ Display goodbye message to user and receive user input """
    if state["finished_conversation"]:
        return END
    else :
        return "agent"
    return state | {"messages": [SystemMessage(content="Goodbye!")]}


In [89]:
# Build StateGraph
register_graph_builder = StateGraph(RegisterState)
register_graph_builder.add_node("agent", welcome_node)
register_graph_builder.add_node("human", human_node)
register_graph_builder.add_edge(START, "agent")
register_graph_builder.add_edge("agent","human")
register_graph_builder.add_conditional_edges("human", user_quit_node)
register_graph = register_graph_builder.compile()

In [ ]:
# Invoke Chat 
from pprint import pprint
config = {"recursion_limit": 100}
user_msg = input("User: ")
messages = [HumanMessage(content=user_msg)]

ai_events = register_graph.invoke({"messages": []},config=config)

for msg in ai_events["messages"]:
    pprint(msg)
    